# Naive Bayes를 사용하여 뉴스 기사 분류

이전 섹션에서 설명한 자연어 처리 기술을 사용하여 텍스트 데이터가 수치 특성으로 변환되면 텍스트 분류는 다른 분류 작업과 동일하게 작동합니다.

## 수입품

In [1]:
%matplotlib inline

from pathlib import Path

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

## 뉴스 기사 분류

우리는 이전과 같이 읽은 BBC 기사를 사용하여 뉴스 기사 분류를 위한 Naive Bayes 모델의 그림으로 시작하여 5개 카테고리에서 2,225개의 기사가 포함된 DataFrame을 얻습니다.

### BBC 기사 읽기

In [2]:
DATA_DIR = Path('..', 'data')

In [3]:
path = DATA_DIR / 'bbc'
files = sorted(list(path.glob('**/*.txt')))
doc_list = []
for i, file in enumerate(files):
    topic = file.parts[-2]
    article = file.read_text(encoding='latin1').split('\n')
    heading = article[0].strip()
    body = ' '.join([l.strip() for l in article[1:]])
    doc_list.append([topic, heading, body])

In [4]:
docs = pd.DataFrame(doc_list, columns=['topic', 'heading', 'body'])
docs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   topic    2225 non-null   object
 1   heading  2225 non-null   object
 2   body     2225 non-null   object
dtypes: object(3)
memory usage: 52.3+ KB


### 계층화된 학습-테스트 분할 만들기

데이터를 기본 75:25 열차-테스트 세트로 분할하여 테스트 세트 클래스가 열차 세트를 밀접하게 반영하는지 확인합니다.

In [5]:
y = pd.factorize(docs.topic)[0]
X = docs.body
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

### 텍스트 데이터 벡터화

훈련 세트에서 어휘를 학습하고 기본 설정으로 CountVectorizer를 사용하여 두 데이터 세트를 변환하여 거의 26,000개의 기능을 얻습니다.

In [6]:
vectorizer = CountVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)

In [7]:
X_train_dtm.shape, X_test_dtm.shape

((1668, 25951), (557, 25951))

### 다중 클래스 Naive Bayes 모델 학습

In [8]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

### 결과 평가

우리는 거의 98%에 달하는 기본 분류기를 찾기 위해 정확도를 사용하여 멀티클래스 예측을 평가합니다.

#### 정확성

In [9]:
accuracy_score(y_test, y_pred_class)

0.9712746858168761

#### 혼란 매트릭스

In [10]:
pd.DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_class))

,0,1,2,3,4
0,120,0,6,0,2
1,0,94,2,0,1
2,1,0,103,0,0
3,0,0,1,127,0
4,0,1,2,0,97
